# Preprocessing Data

-- Land subsidence project: 15 March 2025 --

---

#### Create train/test pickle files for dMG-dPLT

Current Operations:
- Reformat dataframes.
- Keep only minimally available temporal data shared by all sites.

In [1]:
### Paths
# 1. Station information
# 2. Land subsidence  measurements (monthly)
# 3. Groundwater level measurements (monthly)
# 4. Lithology, geological parameters per aquifer layer

station_path = '/projects/mhpi/leoglonz/data/extracted/land_subsidence/all_stations_info_subsidence.csv'
ls_path = '/projects/mhpi/leoglonz/data/extracted/land_subsidence/all_stations_monthly_subsidence.csv'
gw_path = '/projects/mhpi/leoglonz/data/extracted/land_subsidence/all_stations_monthly_water_levels.csv'
geo_path = '/projects/mhpi/leoglonz/data/extracted/land_subsidence/geology_layers.csv'

save_path = '/projects/mhpi/leoglonz/data/extracted/land_subsidence/'

In [4]:
gw['station_count'].sum()

np.int64(6712)

In [2]:
import pandas as pd
import os
import pickle


### Read data
stations = pd.read_csv(station_path)
ls = pd.read_csv(ls_path)
gw = pd.read_csv(gw_path)
geo = pd.read_csv(geo_path)


### Format dataframes
ls = ls.rename(columns=str.lower)
ls['date'] = pd.to_datetime(ls['date'])
ls['date'] = ls['date'].dt.date

gw = gw.rename(columns={
    'Subsidence_Station_ID':'station_id',
    'Subsidence_Station_Name':'station_name',
    'Month':'date',
    }).rename(columns=str.lower)
gw['date'] = pd.to_datetime(gw['date'], format='%Y-%m')
gw['date'] = gw['date'].dt.date

print(f"Available keys:\n ls: {ls.columns}\n gw: {gw.columns}\n geo: {geo.columns}\n")

avail_months = min(len(ls['date'].unique()), len(gw['date'].unique()))
print(f"Available data:\n {len(ls['station_id'].unique())} sites,\n {avail_months} months")

Available keys:
 ls: Index(['vert_mm_avg', 'vert_std_mm_avg', 'vert_mm_first', 'date',
       'change_mm_avg', 'change_mm_first', 'station_id'],
      dtype='object')
 gw: Index(['station_id', 'station_name', 'date', 'wlm_rpe', 'wlm_rpe_change',
       'gwe', 'gwe_change', 'station_count', 'interpolated'],
      dtype='object')
 geo: Index(['station_id', 'station_lat', 'station_lon', 'geology_id', 'distance_m',
       'geology_x_feet', 'geology_y_feet', 'layer_number', 'depth_midpoint',
       'percent_coarse', 'void_ratio', 'compression_index',
       'recompression_index', 'OCR', 'hydraulic_conductivity',
       'coefficient_vol_compressibility', 'initial_effective_stress',
       'thickness', 'is_fine_grained', 'is_clay', 'is_corcoran_clay'],
      dtype='object')

Available data:
 16 sites,
 245 months


In [370]:
### Get time ranges for each site:
print('# --- Available time ranges per site --- #')
time_ranges = []
min_all = pd.Timestamp('2000-01-01').date()
max_all = pd.Timestamp('2100-01-01').date()

for site in ls['station_id'].unique():
    site_ls = ls[ls['station_id'] == site]
    site_gw = gw[gw['station_id'] == site]
    min_date = max(site_ls['date'].min(), site_gw['date'].min())
    max_date = min(site_ls['date'].max(), site_gw['date'].max())
    time_ranges.append([site, min_date, max_date])

    # print(max_all, max_date)
    min_all = max(min_all, min_date)
    max_all = min(max_all, max_date)

    print(f"{site}: {min_date} - {max_date}")

print('# -------------------------------- #')
print(f"Minimum availability: {min_all} - {max_all}")

### Align time ranges
print("\nAligning time ranges...")
ls_aligned = ls[(ls['date'] >= min_all) & (ls['date'] <= max_all)]  # Remove first day of the new range
gw_aligned = gw[(gw['date'] >= min_all) & (gw['date'] <= max_all)]


avail_months = min(len(ls_aligned['date'].unique()), len(gw_aligned['date'].unique()))
print(f"New data dimensions:\n {len(ls_aligned['station_id'].unique())} sites,\n {avail_months} months")

# --- Available time ranges per site --- #
DIXN: 2012-07-01 - 2024-10-01
P208: 2006-04-01 - 2024-03-01
P266: 2005-06-01 - 2024-03-01
P267: 2005-05-01 - 2024-10-01
P268: 2005-05-01 - 2024-09-01
P269: 2008-08-01 - 2024-04-01
P270: 2005-11-01 - 2024-03-01
P271: 2004-07-01 - 2024-09-01
P275: 2006-08-01 - 2024-05-01
P276: 2006-04-01 - 2024-10-01
P336: 2007-10-01 - 2024-10-01
P339: 2007-08-01 - 2024-10-01
P344: 2006-10-01 - 2024-10-01
P345: 2005-11-01 - 2024-10-01
P349: 2006-03-01 - 2024-10-01
VCVL: 2012-10-01 - 2024-11-01
# -------------------------------- #
Minimum availability: 2012-10-01 - 2024-03-01

Aligning time ranges...
New data dimensions:
 16 sites,
 137 months


In [371]:
### Get max number of available layers accross sites:
# Note: later make the other layers 0 thickness if unavailable

min_layer = 100
for site in ls['station_id'].unique():
    geo_station = geo[geo['station_id'] == site]

    min_layer = min(min_layer, geo_station['layer_number'].max()) 

print(f"Trimming all sites to {min_layer} layers")
geo_aligned = geo[geo['layer_number'] <= min_layer]

Trimming all sites to 9 layers


In [372]:
### See missing rows in ls data ###
df1 = gw.copy()
df2 = ls.copy()

# Find rows in df2 that are not in df1
extra_rows = df2[~df2.set_index(['station_id', 'date']).index.isin(df1.set_index(['station_id', 'date']).index)]

# Find rows in df1 that are not in df2
missing_rows = df1[~df1.set_index(['station_id', 'date']).index.isin(df2.set_index(['station_id', 'date']).index)]

print("Extra rows in df2:")
print(extra_rows)

print("Missing rows in df2:")
print(missing_rows)

Extra rows in df2:
      vert_mm_avg  vert_std_mm_avg  vert_mm_first        date  change_mm_avg  \
145    -51.137333         7.369000         -54.81  2024-11-01       4.874925   
146    -51.652903         7.446129         -51.73  2024-12-01      -0.515570   
147    -49.265161         6.541935         -54.25  2025-01-01       2.387742   
357    -12.114667         7.173000         -12.61  2024-04-01      -0.437570   
358    -12.227097         7.264839         -13.65  2024-05-01      -0.112430   
...           ...              ...            ...         ...            ...   
3186     9.541935         7.444194          11.20  2012-07-01            NaN   
3187    12.232903         7.452903           9.54  2012-08-01       2.690968   
3188    14.210333         7.490667           7.75  2012-09-01       1.977430   
3327   -25.059355         7.360968         -29.54  2024-12-01      -6.734355   
3328   -24.675806         6.295161         -28.03  2025-01-01       0.383548   

      change_mm_firs

In [373]:
### Merge data on ls < gw < geo
ls_gw = pd.merge(ls_aligned, gw_aligned, how='left', on=['station_id', 'date'])

data = pd.merge(ls_gw, geo_aligned, how='left', on='station_id') 

data.drop(columns=[
    #### Land subsidence ####
    'vert_mm_avg',
    'vert_std_mm_avg',
    'vert_mm_first',
    'change_mm_avg',
    #### Groundwater ####
    'station_name',
    'wlm_rpe',
    'wlm_rpe_change',
    'gwe',
    'station_count',
    'interpolated',
    #### Geology ####
    'station_lat',
    'station_lon',
    'geology_id',
    'distance_m',
    'geology_x_feet',
    'geology_y_feet',
    'depth_midpoint',
    'is_fine_grained',
    'is_clay',
    'is_corcoran_clay',
], inplace=True)

data.rename(columns={
    'change_mm_first':'subsidence',
    'gwe_change':'gwe',
}, inplace=True)

data = data.sort_values(by=['date', 'station_id', 'layer_number']).reset_index(drop=True)

### Save combined data
data.to_csv(os.path.join(save_path, 'ls_combined.csv'), index=False)

In [374]:
data.head()

,date,subsidence,station_id,gwe,layer_number,percent_coarse,void_ratio,compression_index,recompression_index,OCR,hydraulic_conductivity,coefficient_vol_compressibility,initial_effective_stress,thickness
0,2012-10-01,3.08,DIXN,19.21,1,0,1.193599,0.395733,0.059360,1.749515,9.757353e-10,0.000372,30.772213,8.5344
1,2012-10-01,3.08,DIXN,19.21,2,0,1.184455,0.389637,0.058446,1.679840,9.420891e-10,0.000370,75.048388,3.6576
2,2012-10-01,3.08,DIXN,19.21,3,100,0.781255,0.096251,0.009625,1.077790,8.977119e-06,0.000045,174.025301,13.1064
3,2012-10-01,3.08,DIXN,19.21,4,0,1.142164,0.361443,0.054216,1.391996,8.009529e-10,0.000358,284.659556,26.5176
4,2012-10-01,3.08,DIXN,19.21,5,0,1.118847,0.345898,0.051885,1.254964,7.323944e-10,0.000350,403.628512,4.5720


In [378]:
### Convert data to numpy arrays ###

dates = data['date'].unique()
sites = data['station_id'].unique()
layers = data['layer_number'].unique()

n_dates = len(dates)
n_sites = len(sites)
n_layers = len(layers)

full_index = pd.MultiIndex.from_product(
    [dates, sites, layers],
    names=['date', 'station_id', 'layer_number']
)

data_pivot = data.set_index(['date', 'station_id', 'layer_number'])
data_pivot = data_pivot.reindex(full_index).reset_index()

feature_columns = [col for col in data.columns if col not in ['date', 'station_id', 'layer_number']]
n_features = len(feature_columns)

# Fill missing values with 0
data_pivot[feature_columns] = data_pivot[feature_columns].fillna(0)

data_array = data_pivot[feature_columns].to_numpy().reshape(n_dates, n_sites, n_layers, n_features)
print("Data array shape:", data_array.shape)


# 1. Attributes (geology)
attr_array = data_array[:,:,:,2:]

# 2. Groundwater
forc_array = attr_array[:,:,0,1]

## 3. Target (subsidence)
target_array = data_array[:,:,0,0]


### Save to pickle file
dict = {
    'attributes': attr_array,
    'forcing': forc_array,
    'target': target_array,
}

path = os.path.join(save_path, 'ls_train')
with open(path, 'wb') as f:
    pickle.dump(data_array, f)
    print(f"Data saved to pickle file: {path}")

Data array shape: (137, 16, 9, 11)
Data saved to pickle file: /projects/mhpi/leoglonz/data/extracted/land_subsidence/ls_train
